## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [2]:
import pandas as pd
data = pd.read_csv('/datasets/data.csv') #открываем файл и записываем его в переменную 
data.info()                              #запрашиваем информацию DataFrame  


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


### Вывод

Применив к полученным данным метод .info(), получили информацию о количестве столбцов и строк. В данных мы обнаруживаем 12 столбцов и 21525 строк. Однако, не во всех столбцах мы видим одинаковое число значений. Встречается значение 19531. Это говорит о том, что в некоторых строках есть пропущенные значения. Беглый осмотр названий столбцов говорит нам о том, что с названиями столбцов все в порядке (на одном языке, пробелы отсутствуют, пробелы между словами заменены на подчеркивания). Типы данных встречаются строковые и числовые (целые и вещественные). 

### Шаг 2. Предобработка данных

### Обработка пропусков

In [3]:
data.isnull().sum()
data['days_employed'] = abs(data['days_employed'])       #избавляемся от отрицательных значений в столбце 'days_employed'
data.isnull().sum()                                      #находим столбцы с пропущенными значениями и количество этих значений
data['days_employed'] = data['days_employed'].fillna(0)  #заменяем пропущенные значения в столбце 'days_employed' на 0. 
data['total_income'] = data['total_income'].fillna(0)    #заменяем пропущенные значения в столбце 'total_income' на 0.


### Вывод

Воспользовавшись методом .isnull() и .sum() получаем информацию о количестве нулевых значений NaN. В колонках days_employed и total_income их одинаковое количество - 2174. Думаю, что появление этих пропусков не случайно. Переменные days_employed (общий трудовой стаж в днях) и total_income (ежемесячный доход) скорее всего связаны друг с другом. Именно поэтому отсутствие информации по клиенту в столбце days_employed могло повлечь за собой появление значения NaN в столбце total_income.
Значения в столбцах days_employed и total_income являются числовыми, произведем замену пропущенных значений на 0. Ноль выбран не случайно. Если заменить на какое либо другое число, например на среднее значение (или медиану), то это действие исказит картину при подсчете процента возврата кредита в финальных расчетах, т.к. в действительности нам не известно какой доход был на месте пропущенных значений.

### Замена типа данных

In [4]:
data[['days_employed', 'total_income']] = data[['days_employed', 'total_income']].astype('int') #заменим тип данных float на int
names_of_columns = ['education', 'family_status', 'income_type', 'purpose']      #создаем список с названиями столбцов по которым должен пройти цикл
i = 0                                                                            #создаем счетчик
for i in names_of_columns:                                                       #цикл для приведения всех строковых значений к единому регистру
    data[i] = data[i].str.lower()
    i =+ 1


### Вывод


Изменение типа данных с вещественного (float) на целочисленный (integer) необходимо провести для столбцов days_employed и total_income. Выполним изменения с помощью метода .astype(), т.к. этот метод позволяет просто и быстро (указав нужный тип) изменить тип данных.

### Обработка дубликатов

In [5]:
all_data_columns = data.columns.tolist()                       #заносим названия колонок в список
row_cnt = data.groupby(all_data_columns).size().reset_index()  #группируем по целой строке и считаем кол-во вхождений
row_cnt.rename(columns = {0: 'cnt'}, inplace = True)           #переименовываем столбец с количеством повторов строки
row_cnt.loc[row_cnt['cnt'] > 1]                                #оставляем только те строки, которые повторяются больше 1 раза
data = data.drop_duplicates().reset_index(drop=True)           #удаляем дубликаты
data.duplicated().sum()                                        #проверяем на наличие дубликатов


0

### Вывод

Перед поиском дубликатов приведем все столбцы со строковыми значениями к нижнему регистру методом .str.lower(). Для этого создадим цикл который будет перебирать необходимые нам столбцы и применять к ним метод .str.lower(). После того как все строковые значения приведены к одному регистру можно выполнять поиск дубликатов. Дубликаты ищем методом .duplicated().Поиск дубликатов показывает нам что дубликаты в DataFrame есть. Избавиться от дубликатов можно методом .drop_duplicates(). Из-за того что вместе со строками удаляются и индексы этих строк, необходимо сбросить все старые индексы и назначить новые, это можно сделать применив в очищенной таблице метод .reset_index(drop=True). 

Прежде чем удалить дубликаты я поглядел на строки которые дублируются. Заметил, что дублировалисть только те строки, у которых в столбцах days_employed и total_income были значения NaN (мы их заменяли на 0). Все найденные дубликаты являются "грубыми". Думаю что отсутствие данных в столбцах days_employed и total_income повлияло на дублирование этих строк.

### Лемматизация

In [6]:
from pymystem3 import Mystem
from collections import Counter
m = Mystem()
def purpose_group(text):                              #объявляем функцию, принимающую строки из столбца 'purpose'
    lemmas = (m.lemmatize(text))
    if 'свадьба' in lemmas:                           #прописываем условия. При определенных леммах, функция будет возвращать нужную нас строку 
        return 'торжество'
    if 'автомобиль' in lemmas:
        return 'автомобиль'
    if 'недвижимость' in lemmas:
        return 'операции с недвижимостью'
    if 'жилье' in lemmas:
        return 'операции с недвижимостью'
    if 'образование' in lemmas:
        return 'образование'
data['lemmas'] = data['purpose'].apply(purpose_group) #создаем новый столбец, в который будут записываться нужные нам строки

### Вывод

Для того чтобы категоризировать данные по столбцу 'purpose', не обходимо все имеющиеся строки обработать и разбить на группы. Такую операцию возможно сделать с помощью лемматизации. Из всех имеющихся строк в столбце 'purpose', с помощью лемматизации мы выделим леммы слов. Проанализировав столбец 'purpose' стало ясно что логичнее всего было бы выделить леммы слов 'свадьба', автомобиль', 'недвижимость', 'жилье', 'образование'. Что я и сделал в функции purpose_group. Функция работает по следубщему принципу: принимает на вход строку и лемматизирует ее, записывая полученные леммы в переменную lemmas. Затем проверяет содержащиеся в переменной lemmas леммы на условия. Если какое-то из условий выполняется, функция возвращает строку в соответствии с выполненным условием. 

Для того чтобы в каждой строке исходной таблицы прописать категорию цели, необходимо сделать так, чтобы функция проверяла каждую строку в столбце 'purpose' и записывала возвращаемое значение в новый столбец. Такое поведение возможно реализовать с помощью метода .apply().  
строкой data['lemmas'] = data['purpose'].apply(purpose_group) мы записываем в новый столбец 'lemmas' значения которые нам возвращает функция purpose_group. На вход берутся строки из столбца 'purpose'.

### Категоризация данных

In [7]:
def count_of_children(num):   #функция возвращяющая строку в зависимости от количества детей в семье
    if num == 0:
        return 'нет детей'
    elif num == 1:
        return '1 ребенок'
    return 'многодетные'
data['count_of_children'] = data['children'].apply(count_of_children)  #добавляем новый столбец в таблицу. с помощью метода .apply передаем для функции данные из столбца children 


data_clean = data[ (data['days_employed'] != 0) & (data['total_income'] != 0) ].reset_index(drop=True)  # создаем новую таблицу, очищенную от нулевых значений
def total_income_function(total_income):   # создаем функцию которая делит на категории людей с разным доходом. Предельные значения выбраны условно
    if total_income <= 100000:
        return 'низкий доход'
    elif 100000 < total_income <= 1000000:
        return 'средний доход'
    return 'высокий доход'
data_clean['category_of_incom'] = data['total_income'].apply(total_income_function)  # Добавляем в таблицу новый столбец с категориями людей по доходу
data


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemmas,count_of_children
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,операции с недвижимостью,1 ребенок
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,автомобиль,1 ребенок
2,0,5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,операции с недвижимостью,нет детей
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,образование,многодетные
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,торжество,нет детей
5,0,926,27,высшее,0,гражданский брак,1,M,компаньон,0,255763,покупка жилья,операции с недвижимостью,нет детей
6,0,2879,43,высшее,0,женат / замужем,0,F,компаньон,0,240525,операции с жильем,операции с недвижимостью,нет детей
7,0,152,50,среднее,1,женат / замужем,0,M,сотрудник,0,135823,образование,образование,нет детей
8,2,6929,35,высшее,0,гражданский брак,1,F,сотрудник,0,95856,на проведение свадьбы,торжество,многодетные
9,0,2188,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425,покупка жилья для семьи,операции с недвижимостью,нет детей


### Вывод

Для более наглядного и правильного анализа необходимо категоризировать некоторые данные, например данные о количестве детей, данные о целях и данные о категории по доходу. Столбец с целями мы уже добавили в таблицу, столбцы с количеством детей и категорией по доходу добавляем в этом шаге. С помощью функции с условиями делим граждан на категории 'нет детей', '1 ребенок', 'многодетные'. После чего добавляем новый столбец в таблицу. Таким же образом поступаем для деления по доходу. Доход был условно  разделен на группы 'низкий доход' (<=100 000), 'средний доход' (от 100 000 до 1 000 000) и 'высокий доход' (более 1 000 000). После чего данные категории добавили в таблицу.

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [17]:
#data_pivot = data.pivot_table(index=['count_of_children'],values='debt', aggfunc='sum').sort_values(by='debt', ascending=False) # делаем сводную таблицу по столбцу count_of_children и узнаем количество должников
#data_pivot['counts'] = data.groupby('count_of_children')['debt'].count() #группируем категории людей и высчитываем общее количество обращений за кредитом
#data_pivot['ratio'] = data_pivot['debt'] / data_pivot['counts'] # узнаем процент должников по каждой категории. Делим количество должников на общее количество обращений за кредитом
#data_pivot.sort_values(by='ratio') # сортируем таблицу от более надежных категорий к менее надежным
data_children = data.groupby('count_of_children').agg({'debt':['sum', 'count']})
data_children['ratio'] = data_children['debt']['sum'] / data_children['debt']['count']
data_children.sort_values(by='ratio')

debt            ratio
                    sum  count          
count_of_children                       
нет детей          1063  14091  0.075438
многодетные         234   2555  0.091585
1 ребенок           444   4808  0.092346

### Вывод

Да, зависимость есть. По сводной таблице видим, что семьи без детей возвращяют кредит чаще чем семьи с детьми. Отличие между многодетными и семьями с одним ребенком незначительны

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [9]:
data_pivot = data.pivot_table(index=['family_status'],values='debt', aggfunc='sum').sort_values(by='debt', ascending=False) # делаем сводную таблицу по столбцу family_status и узнаем количество должников
data_pivot['counts'] = data.groupby('family_status')['debt'].count()#группируем категории людей и высчитываем общее количество обращений за кредитом
data_pivot['ratio'] = data_pivot['debt'] / data_pivot['counts']# узнаем процент должников по каждой категории. Делим количество должников на общее количество обращений за кредитом
data_pivot.sort_values(by='ratio')# сортируем таблицу от более надежных категорий к менее надежным

,debt,counts,ratio
family_status,,,
вдовец / вдова,63,959,0.065693
в разводе,85,1195,0.071130
женат / замужем,931,12339,0.075452
гражданский брак,388,4151,0.093471
не женат / не замужем,274,2810,0.097509


### Вывод

 Не вижу зависимости. Вдовцы/вдовы возвращают кредиты чаще, однако не женат/не замужем возвращают кредиты хуже всего.

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [10]:
data_pivot = data_clean.pivot_table(index=['category_of_incom'],values='debt', aggfunc='sum').sort_values(by='debt', ascending=False) # делаем сводную таблицу по столбцу category_of_incom и узнаем количество должников
data_pivot['counts'] = data_clean.groupby('category_of_incom')['debt'].count()#группируем категории людей и высчитываем общее количество обращений за кредитом
data_pivot['ratio'] = data_pivot['debt'] / data_pivot['counts']# узнаем процент должников по каждой категории. Делим количество должников на общее количество обращений за кредитом
data_pivot.sort_values(by='ratio')# сортируем таблицу от более надежных категорий к менее надежным

,debt,counts,ratio
category_of_incom,,,
средний доход,1069,13431,0.079592
низкий доход,499,5898,0.084605
высокий доход,3,22,0.136364


### Вывод

Видим, что люди со средним доходом возвращают кредиты чаще чем люди с высоким и низким доходом.

- Как разные цели кредита влияют на его возврат в срок?

In [11]:
data_pivot = data.pivot_table(index=['lemmas'],values='debt', aggfunc='sum').sort_values(by='debt', ascending=False)# делаем сводную таблицу по столбцу lemmas и узнаем количество должников
data_pivot['counts'] = data.groupby('lemmas')['debt'].count()#группируем категории людей и высчитываем общее количество обращений за кредитом
data_pivot['ratio'] = data_pivot['debt'] / data_pivot['counts']# узнаем процент должников по каждой категории. Делим количество должников на общее количество обращений за кредитом
data_pivot.sort_values(by='ratio')# сортируем таблицу от более надежных категорий к менее надежным

,debt,counts,ratio
lemmas,,,
операции с недвижимостью,782,10811,0.072334
торжество,186,2324,0.080034
образование,370,4013,0.092200
автомобиль,403,4306,0.093590


### Вывод

Видим что операции с недвижимостью наиболее надежные. Кредиты по этой цели возвращают чаще.

### Шаг 4. Общий вывод

По итогам анализа мы выяснили, что для кредитного отдела банка самыми надежными заемщиками будут являться люди без детей, со средним доходом и с целями связанными с недвижимостью. Все остальные категории будут являться более рискованными, т.к по ним процент невозврата выше.

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.